In [14]:
using DataFrames, CSV, Impute, Dates, Plots, Statistics, Interpolations

In [3]:
df = CSV.File("boston.csv") |> DataFrame;

In [4]:
dfdaily = filter(row -> strip(row[:REPORT_TYPE]) == "SOD", df);
dfh = filter(row -> !(strip(row[:REPORT_TYPE]) in ["SOD", "SOM"]), df)
sum(ismissing.(dfh[:, :HourlyDryBulbTemperature]))

0

In [11]:
function clean(x)
    if ismissing(x) || x isa Float64
        return x
    end
    m = match(r"[+-]?([0-9]*[.])?[0-9]+",x)
    return m ==nothing ? missing : parse(Float64, m.match)
end
function resample(dfh, factor)
    dfh[!,factor] = clean.(dfh[:,factor])
    timeseries = Impute.interp(dfh[!, [:DATE, factor]]) |> Impute.locf() |> Impute.nocb() 
    timeseries[!, factor] = convert.(Float64, timeseries[!, factor]) # remove the missing type
    # even sampling
    timeseries[!, :rounded] = map((d) -> round(d, Dates.Hour), timeseries[!, :DATE])
    even_sampling = by(timeseries, :rounded, factor => mean)
    rename!(even_sampling, Symbol(factor, "_mean") => :data)
    return even_sampling
end

resample (generic function with 1 method)

In [12]:
factors = [:HourlyDryBulbTemperature, :HourlyPrecipitation, :HourlySeaLevelPressure];

In [37]:
mdata = resample(dfh, :HourlyDryBulbTemperature)
prec = resample(dfh, :HourlyPrecipitation)
pres = resample(dfh, :HourlySeaLevelPressure)
mdata[!, :prec] = prec[:, :data]
mdata[!, :pres] = pres[:, :data];
rename!(mdata, Dict(:data => :temp, :rounded => :sampleT));

In [34]:
size(mdata).-1

(25053, 3)

In [71]:
monthdayhour = x -> [Dates.month(x) Dates.day(x) Dates.hour(x)];

In [47]:
 mdata[1:5,2:3]

,temp,prec
,Float64,Float64
1,2.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
5,0.0,0.0


In [48]:
reduce(vcat, mdata[1:5,2:3] |> Matrix)'

1×10 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 2.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0

In [86]:
_, ntypes = size(mdata) .- 1
year, month, startday = start
additional_factors = 3                    # for month, day, hour info
historical_factors = 24*dayslookback
ntype_factors = ntypes*historical_factors
total_factors = ntype_factors + additional_factors
n_datapoints = 24*ndays
X = zeros(n_datapoints, total_factors)
y = zeros(n_datapoints);
t0 = DateTime(year, month, startday, 0, 0)
dateidx = findall(x -> x== t0, mdata[:, :sampleT])[1];

In [87]:
hist = reduce(vcat, mdata[dateidx:dateidx+historical_factors-1,2:end] |> Matrix)'

1×72 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 8.0  8.0  8.0  7.0  6.0  5.0  5.0  5.0  …  30.25  30.24  30.23  30.24  30.24

In [88]:
offset=1

1

In [89]:
for idx=1:n_datapoints
    println(idx)
    next_point = dateidx+historical_factors+idx
    predict = next_point+offset-1
    predict_date = mdata[predict,:sampleT]
    X[idx, 1:total_factors] = [hist monthdayhour(predict_date)]
    y[idx] = mdata[predict,predicttype]
    hist = [hist[(1+ntypes):end]' mdata[next_point,2:end]]
    println(hist)
end

1
Any[7.0 6.0 5.0 5.0 5.0 6.0 7.0 9.0 11.0 13.0 14.0 17.0 18.0 18.0 18.0 18.0 18.0 17.0 17.0 16.0 17.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 30.26 30.24 30.25 30.24 30.24 30.25 30.27 30.29 30.31 30.32 30.33 30.31 30.27 30.25 30.23 30.24 30.23 30.23 30.23 30.25 30.24 30.23 30.24 30.24 DataFrameRow
│ Row │ temp    │ prec    │ pres    │
│     │ Float64 │ Float64 │ Float64 │
├─────┼─────────┼─────────┼─────────┤
│ 49  │ 18.0    │ 0.0     │ 30.23   │]
2


DimensionMismatch: DimensionMismatch("tried to assign 1×73 array to 1×75 destination")

In [30]:
#function build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=1)
    _, ntypes = size(mdata) .- 1
    year, month, startday = start
    additional_factors = 3                    # for month, day, hour info
    historical_factors = 24*dayslookback
    ntype_factors = ntypes*historical_factors
    total_factors = ntype_factors + additional_factors
    n_datapoints = 24*ndays
    X = zeros(n_datapoints, total_factors)
    y = zeros(n_datapoints)
    t0 = DateTime(year, month, startday, 0, 0)
    dateidx = findall(x -> x== t0, dataseries[:, :sampleT])[1];
    hist = reduce(vcat, mdata[dateidx:dateidx+historical_factors-1,2:end] |> Matrix)'
    for idx=1:n_datapoints
        next_point = dateidx+historical_factors+idx
        predict = next_point+offset-1
        predict_date = dataseries[predict,:sampleT]
        X[idx, 1:total_factors] = [hist monthdayhour(predict_date)]
        y[idx] = dataseries[predict,predicttype]
        hist = [hist[(1+ntypes):end]' dataseries[next_point,predicttype]]
    end
#    return X,y
#end

,rounded,temp,prec,pres
,DateTime,Float64,Float64,Float64
1,2018-01-01T01:00:00,2.0,0.0,30.2
2,2018-01-01T02:00:00,1.0,0.0,30.21
3,2018-01-01T03:00:00,1.0,0.0,30.21
4,2018-01-01T04:00:00,1.0,0.0,30.2
5,2018-01-01T05:00:00,0.0,0.0,30.19
6,2018-01-01T06:00:00,0.0,0.0,30.2
7,2018-01-01T07:00:00,0.0,0.0,30.22
8,2018-01-01T08:00:00,0.0,0.0,30.24
9,2018-01-01T09:00:00,1.0,0.0,30.24


In [73]:
start = 2018, 1, 2
dayslookback = 1
ndays = 10
predicttype = :temp

:temp